In [1]:
import streamlit as st
import pandas as pd
import numpy as np


import nltk, re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


In [2]:
data = pd.read_csv('./netflix_titles.csv')
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [3]:
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jonat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def preprocess(data_input,column_name,language="english",tolower=True, html_tags=True, stemming = False,stop_words=False):
    """ Limpieza de datos,se remueven caracteres especiales,limpieza de tags de  HTML
        se convierte a munisculas el texto,stemming de texto, se retiras stopwords
        se elimina doble espacio"""
     
    stemmer_preprocces = SnowballStemmer(language=language)
    stop=set(stopwords.words(language))
    
       
    #remove special characters
    temp = data_input[column_name].copy()

    #elimina carcateres especiales y los convierte en string
    temp = temp.apply(lambda x: re.sub("[^A-Za-z0-9-\s]+","",str(x))) 
           
    #clean HTML tags
     # re sub re compile extraera las etiquetas <> las reemplazara por un espacio en blanco y se las asignara a x 
    if html_tags: temp = temp.apply( lambda x: re.sub(re.compile('<.*?>'), '', x) )
         
    #tolower todo en minusculas
    if tolower: temp =temp.apply(lambda x: str(x).lower())     
    
    #stemming
    #reducira las palabras a su origen reduciendo la dimension de la palabra
    ##extraera solo los vocablos o prefijos
    if stemming: temp = temp.apply( lambda x: ' '.join([stemmer_preprocces.stem(x_i) for x_i in x.split(' ')]) )
                
    #remove stopwords
    if stop_words: temp = temp.apply( lambda x: ' '.join([y for y in x.split(' ') if y not in stop]) )
        
    #remove double space
    result = temp.apply(lambda x: re.sub("\s\s+" , " ", x))   
         
    return result 

In [5]:
#funcion coseno de similitud
def cosine_similarity(vector1, vector2):
    vector1 = np.array(vector1)
    vector2 = np.array(vector2)
    return np.dot(vector1, vector2) / (np.sqrt(np.sum(vector1*2)) * np.sqrt(np.sum(vector2*2)))

In [6]:
def mvp_recommender(user_input):
    df_sample_title = pd.DataFrame([user_input])

    preprocessed_sample = preprocess(df_sample_title,
                                       column_name=[0],\
                                       language='english',\
                                       html_tags=False,\
                                       stemming=True,\
                                       stop_words = True)
    
     #devuelve un vector con una distancia como array para preprocessed sample
    bow_sample = name_bow_vectorizer.transform(preprocessed_sample).toarray()
    
    #se almacenara j es [0 ]y coseno de similitud [1]
    temp_similarity = []
    
    for j in range(0, name_bow_tfidf.shape[0]): 
        temp_similarity.append([j, cosine_similarity(bow_sample, name_bow_tfidf.toarray()[j]) [0]])
        #creara dos columnas una con los indices y otra con el coseno de similitud
        #[0] regresara solo un numero 

    #valores mas altos de similitud
    similar_courses = pd.DataFrame(temp_similarity).sort_values(by=[1],ascending=False)
    
    #similitudes mas altas
    similar_coursesl = similar_courses.loc[similar_courses[1]>0].head(5)#regresara las 5 primeras
    
    #Matriz de resultados concatenados con similitudes
    result=similar_coursesl.merge(data['title'],left_on=0,right_index=True, how='left')
             
    return result    

In [7]:
#se llamara la funcion preprocess y se le aplicaran las funciones definidas en ella
preprocessed_summary= preprocess(data, column_name='description', language='english',html_tags=False,
                                   stemming=True, stop_words = True)

#norm puede ser l1 o l2, ngramas de 1 a 2, genera una matriz de entidades de TF-idf 
name_bow_vectorizer = TfidfVectorizer(norm="l2", analyzer='word', ngram_range=(1,2), max_features = 500)

#se aplica funcion a los datos procesados
name_bow_tfidf = name_bow_vectorizer.fit_transform(preprocessed_summary) 


In [9]:
interact(mvp_recommender, user_input='war nations')

interactive(children=(Text(value='war nations', description='user_input'), Output()), _dom_classes=('widget-in…

<function __main__.mvp_recommender(user_input)>